In [40]:
import video_dataset
import matplotlib.pyplot as plt

from skimage import io
from importlib import reload
from IPython.core.debugger import set_trace

In [57]:
video_directory = r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs'
samples = video_dataset.VideoDataset(video_directory, 10)

In [60]:
reload(video_dataset)

<module 'video_dataset' from 'C:\\Users\\John\\PythonVenvs\\VideoSuperResolution\\Scripts\\video_dataset.py'>